### P1-Preprocesamiento

    *Tokenizacion 
    *Eliminacion de signos de puntuacion
    *Eliminacion de Stopwords
    *Lematizacion
    *Stemming
    *Normalizar (minusculas)
    *Bag of Words
    
    

#### Importamos las librerias a utilizar 

In [1]:
from unidecode import unidecode
import pandas as pd

import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

import spacy
nlp = spacy.load('es_core_news_sm')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ianca\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ianca\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
texto = pd.read_csv("./datasets/sentiment_analysis_dataset.csv")
texto = texto["text"]

doc_ejemplos = [
    'este es el primer documento',
    'este documento es el segundo documento',
    'y este es el tercer documento',
    'es este el primer documento'
]


#### Creamos la funcion de tokenizar

In [3]:
def tokenize_text(text):
    tokens = []
    current_word = ''

    for char in text:
        if char.isalnum() or char == "'":
            current_word += char
        elif current_word:
            tokens.append(current_word.lower())
            current_word = ''
    
    if current_word:
        tokens.append(current_word.lower())

    return tokens

allTokens = []
tokenEjm = []

for row in texto:
    tokens = tokenize_text(row)    
    allTokens.append(tokens)
    
for row in doc_ejemplos:
    tokens2 = tokenize_text(row)    
    tokenEjm.append(tokens2)
    
print(tokenEjm) #Funciona


[['este', 'es', 'el', 'primer', 'documento'], ['este', 'documento', 'es', 'el', 'segundo', 'documento'], ['y', 'este', 'es', 'el', 'tercer', 'documento'], ['es', 'este', 'el', 'primer', 'documento']]


#### Definimos nuestras Stopwords en un arreglo

In [4]:
stopwords_espanol = [
    "de", "la", "que", "el", "en", "y", "a", "los", "del", "se", "las", "por", "un", "para", "con", "no",
    "una", "su", "al", "lo", "como", "más", "pero", "sus", "le", "ya", "o", "este", "sí", "porque", "esta",
    "entre", "cuando", "muy", "sin", "sobre", "también", "me", "hasta", "hay", "donde", "quien", "desde",
    "todo", "nos", "durante", "todos", "uno", "les", "ni", "contra", "otros", "ese", "eso", "ante", "ellos",
    "e", "esto", "mí", "antes", "algunos", "qué", "unos", "yo", "otro", "otras", "otra", "él", "tanto",
    "esa", "estos", "mucho", "quienes", "nada", "muchos", "cual", "poco", "ella", "estar", "estas", "algunas",
    "algo", "nosotros", "mi", "mis", "tú", "te", "ti", "tu", "tus", "ellas", "nosotras", "vosotros", "vosotras",
    "os", "mío", "mía", "míos", "mías", "tuyo", "tuya", "tuyos", "tuyas", "suyo", "suya", "suyos", "suyas",
    "nuestro", "nuestra", "nuestros", "nuestras", "vuestro", "vuestra", "vuestros", "vuestras", "esos", "esas",
    "estoy", "estás", "está", "estamos", "estáis", "están", "esté", "estés", "estemos", "estéis", "estén", "estaré",
    "estarás", "estará", "estaremos", "estaréis", "estarán", "estaría", "estarías", "estaríamos", "estaríais",
    "estarían", "estaba", "estabas", "estábamos", "estabais", "estaban", "estuve", "estuviste", "estuvo", "estuvimos",
    "estuvisteis", "estuvieron", "estuviera", "estuvieras", "estuviéramos", "estuvierais", "estuvieran", "estuviese",
    "estuvieses", "estuviésemos", "estuvieseis", "estuviesen", "estando", "estado", "estada", "estados", "estadas",
    "estad", "he", "has", "ha", "hemos", "habéis", "han", "haya", "hayas", "hayamos", "hayáis", "hayan", "habré",
    "habrás", "habrá", "habremos", "habréis", "habrán", "habría", "habrías", "habríamos", "habríais", "habrían",
    "había", "habías", "habíamos", "habíais", "habían", "hube", "hubiste", "hubo", "hubimos", "hubisteis", "hubieron",
    "hubiera", "hubieras", "hubiéramos", "hubierais", "hubieran", "hubiese", "hubieses", "hubiésemos", "hubieseis",
    "hubiesen", "habiendo", "habido", "habida", "habidos", "habidas", "soy", "eres", "es", "somos", "sois", "son",
    "sea", "seas", "seamos", "seáis", "sean", "seré", "serás", "será", "seremos", "seréis", "serán", "sería", "serías",
    "seríamos", "seríais", "serían", "era", "eras", "éramos", "erais", "eran", "fui", "fuiste", "fue", "fuimos",
    "fuisteis", "fueron", "fuera", "fueras", "fuéramos", "fuerais", "fueran", "fuese", "fueses", "fuésemos",
    "fueseis", "fuesen", "sintiendo", "sentido", "sentida", "sentidos", "sentidas", "siente", "sentid", "tengo",
    "tienes", "tiene", "tenemos", "tenéis", "tienen", "tenga", "tengas", "tengamos", "tengáis", "tengan", "tendré",
    "tendrás", "tendrá", "tendremos", "tendréis", "tendrán", "tendría", "tendrías", "tendríamos", "tendríais",
    "tendrían", "tenía", "tenías", "teníamos", "teníais", "tenían", "tuve", "tuviste", "tuvo", "tuvimos", "tuvisteis",
    "tuvieron", "tuviera", "tuvieras", "tuviéramos", "tuvierais", "tuvieran", "tuviese", "tuvieses", "tuviésemos",
    "tuvieseis", "tuviesen", "teniendo", "tenido", "tenida", "tenidos", "tenidas", "tened"
]



#### Creamos la funcion de filtrado de stopwords

In [5]:
def filtrar_stopwords(tokens, stopwords):
    tokens_filtrados = [token for token in tokens if len(token) > 3 and token.lower() not in stopwords]
    return tokens_filtrados

allTokensFiltrados = []
tokenStop = []

for token in allTokens:
    tokensFiltrados = filtrar_stopwords(token, stopwords_espanol)
    allTokensFiltrados.append(tokensFiltrados)

#print(allTokensFiltrados) #Funciona
for row in tokenEjm:
    tokens2 = filtrar_stopwords(row, stopwords_espanol)    
    tokenStop.append(tokens2)
    
print(tokenStop) #Funciona

[['primer', 'documento'], ['documento', 'segundo', 'documento'], ['tercer', 'documento'], ['primer', 'documento']]


#### Creamos la funcion para elimiar acentos 

In [6]:
def quitar_acentos(tokens):
    tokensSinAcentos = [unidecode(token) for token in tokens]
    return tokensSinAcentos


allTokensSinAcentos = []
tokenSin = []
for token in allTokensFiltrados:
    tokensSinAcentos = quitar_acentos(token)
    allTokensSinAcentos.append(tokensSinAcentos)
    
for token in tokenStop:
    tokensSinAcentos = quitar_acentos(token)
    tokenSin.append(tokensSinAcentos)
    
#print(allTokensSinAcentos) #Funciona
print(tokenSin) #Funciona


[['primer', 'documento'], ['documento', 'segundo', 'documento'], ['tercer', 'documento'], ['primer', 'documento']]


#### Creamos a funcion que me lematiza los tokens

In [7]:
def lematizar(tokens):
    lemas = []
    for token in tokens:
        doc = nlp(token)
        for token in doc:
            lemas.append(token.lemma_)
    return lemas

allTokensLematizados = []
tokenlem = []

for token in allTokensSinAcentos:
    lemas = lematizar(token)
    allTokensLematizados.append(lemas)
    
for token in tokenSin:
    lemas = lematizar(token)
    tokenlem.append(lemas)

#print(allTokensLematizados) #Funciona
print(tokenlem) #Funciona


[['primero', 'documento'], ['documento', 'segundo', 'documento'], ['tercer', 'documento'], ['primero', 'documento']]


#### Creamos la funcion que nos hace el Stemming

In [8]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')

def stemming(tokens):
    stems = [stemmer.stem(token) for token in tokens]
    return stems

allTokensStemming = []
tokenStem = []

for token in allTokensSinAcentos:
    raices = stemming(token)
    allTokensStemming.append(raices)
    
for token in tokenSin:
    raices = stemming(token)
    tokenStem.append(raices)
    
#print(allTokensStemming) #Funciona
print(tokenStem) #Funciona


[['prim', 'document'], ['document', 'segund', 'document'], ['terc', 'document'], ['prim', 'document']]


#### Creamos la funcion de las bag of words para vectorizar

In [11]:
def bag_of_words(documents):
    # Creamos un conjunto para almacenar todas las palabras únicas en los documentos
    vocabulary = set()
    for document in documents:
        vocabulary.update(document)

    # Creamos un diccionario para almacenar las frecuencias de las palabras en cada documento
    bow = []
    for document in documents:
        doc_bow = {}
        for word in vocabulary:
            doc_bow[word] = document.count(word)
        bow.append(doc_bow)

    return bow, list(vocabulary)

# Ejemplo de uso:

bow_representation, vocabulary = bag_of_words(allTokensLematizados)

nombre_archivo = "resultados-bag-of-words.txt"

with open(nombre_archivo, "w") as archivo:
    archivo.write("Bag of Words representation:\n")
    for doc_bow in bow_representation:
        archivo.write(str(doc_bow) + "\n")
    archivo.write("\nVocabulary: " + str(vocabulary))

bow_representation2, vocabulary2 = bag_of_words(tokenlem)

print("Bag of Words representation:")
for doc_bow in bow_representation2:
    print(doc_bow)
print("\nVocabulary:", vocabulary2)


# Creamos un diccionario para almacenar la frecuencia de cada palabra en todo el corpus
frecuencia_palabras = {}

# Recorremos todos los documentos en el primer conjunto de datos
for doc_bow in bow_representation:
    for palabra, frecuencia in doc_bow.items():
        if palabra in frecuencia_palabras:
            frecuencia_palabras[palabra] += frecuencia
        else:
            frecuencia_palabras[palabra] = frecuencia

# Recorremos todos los documentos en el segundo conjunto de datos
for doc_bow in bow_representation2:
    for palabra, frecuencia in doc_bow.items():
        if palabra in frecuencia_palabras:
            frecuencia_palabras[palabra] += frecuencia
        else:
            frecuencia_palabras[palabra] = frecuencia

nombre_archivo_frecuencia = "BOW.txt"

with open(nombre_archivo_frecuencia, "w") as archivo_frecuencia:
    archivo_frecuencia.write("Frecuencia de palabras en todo el corpus:\n")
    for palabra, frecuencia in frecuencia_palabras.items():
        archivo_frecuencia.write(f"{palabra}: {frecuencia}\n")

print(f"Se ha guardado la frecuencia de palabras en el archivo: {nombre_archivo_frecuencia}")


Bag of Words representation:
{'documento': 1, 'primero': 1, 'segundo': 0, 'tercer': 0}
{'documento': 2, 'primero': 0, 'segundo': 1, 'tercer': 0}
{'documento': 1, 'primero': 0, 'segundo': 0, 'tercer': 1}
{'documento': 1, 'primero': 1, 'segundo': 0, 'tercer': 0}

Vocabulary: ['documento', 'primero', 'segundo', 'tercer']
Se ha guardado la frecuencia de palabras en el archivo: BOW.txt


#### Preprocesamiento final

In [10]:
archivo_csv_original = './datasets/sentiment_analysis_dataset.csv'
archivo_csv_salida = './datasets/sentiment_analysis_dataset_modificado.csv'

df_original = pd.read_csv(archivo_csv_original)

documentos_texto = [" ".join(tokens) for tokens in allTokensLematizados]

df_original['text'] = documentos_texto
df_original.to_csv(archivo_csv_salida, index=False)

print(df_original['text'])

0                          terminar bien abrumar despues 
1                                          sentir abrumar
2       sentir abrumar cantidad cosa querer dibujar ju...
3       salvador unicar persona abrumar versión nadiec...
4                           denmir helado ar full abrumar
                              ...                        
2585    poder vivir miedo manejen borracho dejar usar ...
2586                           vida constante miedo exito
2587    esquizofrenia mente dividido miedo realidad pa...
2588               miedo como desaparez mundo temer mundo
2589               saltar apocir pile agua miedo relajado
Name: text, Length: 2590, dtype: object
